In [1]:
from parameters.dataset import BPI2012Parameters
from dataset import BPI2012Dataset
from utils.print import print_block
from utils.bpi2012 import remove_trail_steps, print_model_prediction_result, remove_tags_for_seq
import tensorflow as tf
from model import LSTMPredNextAmountSpecificModel, LSTMScenarioCfWithResourceModel
import numpy as np
from dice import EventLogDiCE

In [2]:
dataset = BPI2012Dataset(BPI2012Parameters()) 


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [3]:
trace_idx = 52
_, example_activities, _, example_resources, example_amount, _ = dataset.collate_fn([trace_idx])
print_block(dataset.activity_vocab.list_of_index_to_vocab_2d(example_activities),"Activity")
print_block(dataset.resource_vocab.list_of_index_to_vocab_2d(example_resources),"Resource")


====================Activity====================
| [['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'A_ACCEPTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_SELECTED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'O_SENT_BACK_COMPLETE', 'W_Nabellen offertes_COMPLETE', 'O_ACCEPTED_COMPLETE', 'A_APPROVED_COMPLETE', 'A_REGISTERED_COMPLETE', 'A_ACTIVATED_COMPLETE', 'W_Valideren aanvraag_COMPLETE']] 

====================Resource====================
| [['<SOS>', '112', '112', '112', '11180', '11201', '11201', '11201', '11201', '11201', '11201', '11201', '11049', '11049', '10629', '10629', '10629', '10629', '10629']] 


In [4]:
## Remove tails
tail_length_to_remove = 15
example_idx_activities, example_idx_resources = remove_trail_steps(example_activities, example_resources, tail_length_to_remove)
example_vocab_activities = dataset.activity_vocab.list_of_index_to_vocab_2d(example_idx_activities)[0]
example_vocab_resources = dataset.resource_vocab.list_of_index_to_vocab_2d(example_idx_resources)[0]
print_block(example_vocab_activities, "Activity (Tail removed)")
print_block(example_vocab_resources, "Resource (Tail removed)")


====================Activity (Tail removed)====================
| ['<SOS>', 'A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Resource (Tail removed)====================
| ['<SOS>', '112', '112', '112'] 


In [5]:
pred_model = LSTMPredNextAmountSpecificModel.load(
    "./SavedModels/%s" % (
    "0.8159_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-06-29 18:25:07.216254" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.8159_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-06-29 18:25:07.216254  

| Vocab loaded successfully from: ./SavedModels/0.8159_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-06-29 18:25:07.216254  

| Model loaded successfully from: ./SavedModels/0.8159_LSTMPredNextAmountSpecificModel_AOW_CompleteOnly_2021-06-29 18:25:07.216254  


In [6]:
_ = pred_model(**pred_model.get_example_input())
pred_model.summary()

Model: "LSTMPredNextAmountSpecificModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  832       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm (LSTM)                  multiple                  24832     
_________________________________________________________________
lstm_1 (LSTM)                multiple                  49408     
_________________________________________________________________
sequential (Sequential)      (1, 64)                   4544      
_________________________________________________________________
sequential_1 (Sequential)    (1, 1, 26)                15066     
Total params: 103,898
Trainable params: 103,258
Non-trainable params: 640
___________________________

In [7]:
##### Get model output
example_activities_input = tf.constant(example_idx_activities)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

predicted_df = print_model_prediction_result(pred_model, example_activities_input, example_resources_input, example_amount_input)


========================================Predict result========================================
| Predicted activity with highest probability (0.56) is "W_Completeren aanvraag_COMPLETE" 



<PAD>                                      4.451259e-06
<EOS>                                      9.926849e-06
<SOS>                                      4.039804e-06
A_ACCEPTED_COMPLETE                        3.796275e-01
A_ACTIVATED_COMPLETE                       6.188364e-06
A_APPROVED_COMPLETE                        4.494266e-07
A_CANCELLED_COMPLETE                       3.891121e-02
A_DECLINED_COMPLETE                        2.339571e-02
A_FINALIZED_COMPLETE                       2.688190e-05
A_PARTLYSUBMITTED_COMPLETE                 3.910440e-07
A_PREACCEPTED_COMPLETE                     9.875686e-05
A_REGISTERED_COMPLETE                      4.097941e-07
A_SUBMITTED_COMPLETE                       2.499174e-06
O_ACCEPTED_COMPLETE                        1.308198e-06
O_CANCELLED_COMPLETE      

In [8]:
predicted_df

,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,...,O_DECLINED_COMPLETE,O_SELECTED_COMPLETE,O_SENT_BACK_COMPLETE,O_SENT_COMPLETE,W_Afhandelen leads_COMPLETE,W_Beoordelen fraude_COMPLETE,W_Completeren aanvraag_COMPLETE,W_Nabellen incomplete dossiers_COMPLETE,W_Nabellen offertes_COMPLETE,W_Valideren aanvraag_COMPLETE
0,0.000004,0.00001,0.000004,0.379627,0.000006,4.494266e-07,0.038911,0.023396,0.000027,3.910440e-07,...,2.273254e-07,0.000049,4.700954e-08,0.000007,0.000164,0.000037,0.557637,0.000004,0.000004,0.000002


In [9]:
no_need_tags = ['<EOS>', '<SOS>', '<PAD>']

possible_activities = [ a for a in list(pred_model.activity_vocab.vocabs) if  not a in no_need_tags]
possible_resources =  [ r for r in list(pred_model.resource_vocab.vocabs) if  not r in no_need_tags]

example_vocab_activities_no_tag = remove_tags_for_seq(example_vocab_activities, no_need_tags)

example_vocab_resources_no_tag = remove_tags_for_seq(example_vocab_resources, no_need_tags)

# transform to possible dim
example_idx_activities_no_tag = [ possible_activities.index(v)   for v in example_vocab_activities_no_tag ]

example_idx_resources_no_tag = [
possible_resources.index(v)   for v in example_vocab_resources_no_tag    
]

example_trace_len_no_tag = len(example_vocab_activities_no_tag)
#### Determine feature names for DiCE ####
activity_feature_names = np.array(["activity_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
resource_feature_names = np.array(["resource_step_%d" % (i+1) for i in range(example_trace_len_no_tag)])
possbile_amount = [min(dataset.df["amount"]), max(dataset.df["amount"])]

print_block(example_vocab_activities_no_tag ,title="Example activities without tags", num_marks = 30)
print_block(example_vocab_resources_no_tag ,title="Example resources without tags", num_marks = 30)
print_block(example_amount, title="=================Amount=================", num_marks=16)


==============================Example activities without tags==============================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

==============================Example resources without tags==============================
| ['112', '112', '112'] 

=================================Amount=================================
| [15500.0] 


In [10]:
scenario_model = LSTMScenarioCfWithResourceModel.load(
    "./SavedModels/%s" % ("0.9855_LSTMScenarioCfWithResourceModel_AOW_CompleteOnly_20_times_2021-06-29 10:12:02.918758" 
    )
)


| Model parameters loaded successfully from: ./SavedModels/0.9855_LSTMScenarioCfWithResourceModel_AOW_CompleteOnly_20_times_2021-06-29 10:12:02.918758  

| Vocab loaded successfully from: ./SavedModels/0.9855_LSTMScenarioCfWithResourceModel_AOW_CompleteOnly_20_times_2021-06-29 10:12:02.918758  

| Model loaded successfully from: ./SavedModels/0.9855_LSTMScenarioCfWithResourceModel_AOW_CompleteOnly_20_times_2021-06-29 10:12:02.918758  


In [11]:
_ = scenario_model(**scenario_model.get_example_input())
scenario_model.summary()

Model: "LSTMScenarioCfWithResourceModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  832       
_________________________________________________________________
embedding_3 (Embedding)      multiple                  9216      
_________________________________________________________________
lstm_2 (LSTM)                multiple                  24832     
_________________________________________________________________
lstm_3 (LSTM)                multiple                  49408     
_________________________________________________________________
sequential_2 (Sequential)    (1, 1, 1)                 9157      
Total params: 93,445
Trainable params: 93,059
Non-trainable params: 386
_________________________________________________________________


In [12]:
dice = EventLogDiCE(
    dataset.activity_vocab,
    dataset.resource_vocab,
    possbile_amount,
    possible_activities,
    possible_resources, 
    pred_model,
    scenario_model
    )

In [13]:
cf_out = dice.run_pls(
    ## Input
    example_amount_input.numpy(),
    example_idx_activities_no_tag,
    example_idx_resources_no_tag,
    desired_vocab = "A_DECLINED_COMPLETE",
    
    ## Weight
    class_loss_weight = 1,
    scenario_weight = 200,
    distance_loss_weight = 1e-8,
    cat_loss_weight = 1e-3,
    
    ## Training parameters
    scenario_threshold = 0.5,
    max_iter=600,
    lr=0.5,

    ## Updating fields
    updating_amount_cf=True,
    updating_activity_cf=False,
    updating_resource_cf=True,
    
    ## Options
    use_valid_cf_only=False,
    use_sampling=True,
    class_using_hinge_loss=False,
    scenario_using_hinge_loss=False,
    use_clipping=True, 
)


====================Model Prediction====================
| Prediction: [W_Completeren aanvraag_COMPLETE(22)] | Desired: [A_DECLINED_COMPLETE(7)] 

====================Counterfactual Process====================
| [0] ==========> [1] 

====================! Counterfactual with invalid scenario found in step [2] 🙈 !====================
| Running time: 0.21 

====================Input Amount====================
| [15500.] 

====================Input Activities====================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Input Resource====================
| ['112', '112', '112'] 

====================Valid CF Amount====================
| 15499.0 

====================Valid CF Activities====================
| ['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'] 

====================Valid CF Resource====================
| ['11254', '10933', '10188'] 

====================Valid CF scenario output=====